In [4]:
import pandas as pd
from haversine import haversine

In [9]:
accident_df=pd.read_csv("data 경도 위도 매핑/전주_교통사고_통계.csv")
kinder_df=pd.read_csv("data 경도 위도 매핑/전주시_어린이집.csv")
school_df=pd.read_csv("data 경도 위도 매핑/전주_유초중고_총통계.csv")
bus_station_df=pd.read_csv("data 경도 위도 매핑/전주시_버스 정류장.csv")
gas_station_df=pd.read_csv("data 경도 위도 매핑/전주_충전소_주유소.csv")
## 추가추가

In [10]:
accident_df.head(3)

,Unnamed: 0,Unnamed: 0.1,사고번호,사고일시,요일,시군구,도로명,주소,위도,경도
0,0,0,2.019060e+15,2019년 6월 6일 19시,목요일,전라북도 전주시 완산구 중앙동1가,충경로,전라북도 전주시 완산구 중앙동1가충경로,35.818018,127.145452
1,1,1,2.019090e+15,2019년 9월 30일 13시,월요일,전라북도 전주시 완산구 중앙동1가,전주객사2길,전라북도 전주시 완산구 중앙동1가전주객사2길,35.817335,127.141453
2,2,2,2.019050e+15,2019년 5월 6일 07시,월요일,전라북도 전주시 완산구 중앙동1가,전주객사2길,전라북도 전주시 완산구 중앙동1가전주객사2길,35.817335,127.141453


In [13]:
bus_station_df.head(3)

,설치 지점명,지역(구),지역(동),세부주소,행정동,위도,경도
0,우아롯데아파트,덕진구,우아동,우아동 2동 887,우아2동,35.832510,127.167824
1,여의동우체국전북은행,덕진구,여의동,여의동2가 696,여의동,35.871416,127.076584
2,기린대로경제통상진흥원,덕진구,팔복동,팔복동2가 550,팔복동,35.859761,127.092527


In [14]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,전라북도,직영충전소,충전소,전북 전주시 덕진구 혁신로 360번지,35.864542,127.069401
1,전라북도,(유)삼영에너지,충전소,전북 전주시 완산구 쑥고개로 11 (용복동),35.790810,127.068859
2,전라북도,(유)푸른기업충전소,충전소,전북 전주시 완산구 모악로 4483 (석구동),35.786460,127.128841


## 가산

가능 : 타 주유소, 교통량(교통 사고), 버스 정류장

working on : 차고지 위치

In [38]:
# 거리 총합 리스트(버스)
def 거리총합리스트(data1, data2, name_col):
    distance_list=[]

    for i in range(len(data1)):
        total=0
        for j in range(len(data2)):
            total+=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
        distance_list.append((data1.loc[i][name_col],total))
        
    distance_list=sorted(distance_list,key=lambda x:x[1])
    
    return distance_list[:10]

In [39]:
distance_list=거리총합리스트(gas_station_df, bus_station_df, '상호')
distance_list

[('목원주유소', 1932.117595800026),
 ('(유)피노키오주유소', 1951.6820527794473),
 ('어은터널주유소', 1952.5430694432293),
 ('OK셀프주유소', 1960.7183121518026),
 ('SK에너지㈜ 대광제일주유소', 1964.5835598249605),
 ('대림싼주유소', 1989.9465029210273),
 ('(유)그린피앤엘 시온주유소', 2007.126415917588),
 ('두리주유소', 2032.5060271229088),
 ('대학로self주유소', 2086.5716975460305),
 ('SK행복충전 호남가스충전소', 2097.3811336500444)]

In [40]:
# p 반경 이내 개수 리스트(타 주유소)
def 반경이내_개수_리스트(p, data1, data2, name_col):
    count_list=[]
    for i in range(len(data1)):
        count=0
        for j in range(len(data2)):
            dist=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
            if dist<=p:
                count+=1
        count_list.append((data1.loc[i][name_col],count-1))# 자기 자신 제외
    count_list=sorted(count_list, key=lambda x: x[1], reverse=True)
    return count_list[:10]
   

In [41]:
주유소_count_list=반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')
주유소_count_list

[('동산LPG충전소', 34),
 ('통큰셀프주유소', 33),
 ('첨단주유소', 33),
 ('CJ대한통운㈜ 공단주유소', 31),
 ('팔복주유소', 31),
 ('그린주유소', 31),
 ('유진주유소', 31),
 ('세진셀프주유소', 31),
 ('SK행복충전 대성엘피지충전소', 30),
 ('직영충전소', 29)]

In [46]:
교통사고_count_list=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
교통사고_count_list

[('OK셀프주유소', 690),
 ('(유)피노키오주유소', 650),
 ('목원주유소', 646),
 ('한옥마을주유소', 639),
 ('(유)그린피앤엘 시온주유소', 631),
 ('두리주유소', 609),
 ('한흥주유소', 580),
 ('어은터널주유소', 574),
 ('새로운주유소', 568),
 ('스타주유소', 561)]

## 감점

가능 : 어린이집, 유초중고(2km)

working on : 화재

In [43]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,전라북도,직영충전소,충전소,전북 전주시 덕진구 혁신로 360번지,35.864542,127.069401
1,전라북도,(유)삼영에너지,충전소,전북 전주시 완산구 쑥고개로 11 (용복동),35.790810,127.068859
2,전라북도,(유)푸른기업충전소,충전소,전북 전주시 완산구 모악로 4483 (석구동),35.786460,127.128841


In [44]:
school_df.head(3)

,시도,행정구,교육\n(지원)청,학교급,고등학교\n유형,KEDI 학제,학교명,우편번호,주소,위도,경도
0,전북,전주시,전북,고등학교,일반고,일반고등학교,전북대학교사범대학부설고등학교,54896,전라북도 전주시 덕진구 백제대로 567(금암동. 전북대학교),35.846732,127.129367
1,전북,전주시,전북,고등학교,일반고,일반고등학교,전라고등학교,54836,전라북도 전주시 덕진구 솔내7길 25(송천동1가. 전라고등학교),35.859966,127.124891
2,전북,전주시,전북,고등학교,일반고,일반고등학교,전주고등학교,55007,전라북도 전주시 완산구 권삼득로 2(중노송동. 전주고등학교),35.825388,127.151833


In [45]:
kinder_df.head(3)

,Unnamed: 0,어린이집명,주소,위도,경도
0,0,e편한세상어린이집,"전라북도 전주시 완산구 여울로 161 관리동(서신동, e편한세상)",35.839530,127.106910
1,1,NPS국민연금어린이집,전라북도 전주시 덕진구 기지로 180,35.837661,127.071757
2,2,SK뷰 생글어린이집,"전라북도 전주시 완산구 태평2길 22 관리동(태평동, 태평SKVIEW)",35.824693,127.139501


In [47]:
유치원_count_list=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')
유치원_count_list

[('진영주유소', 162),
 ('삼천주유소', 159),
 ('전주제일주유소', 154),
 ('SK에너지㈜ 코끼리주유소', 145),
 ('룰루랄라주유소', 143),
 ('안행주유소', 142),
 ('유연로주유소', 142),
 ('SK강변주유소', 142),
 ('지에스칼텍스(주)글로리주유소', 142),
 ('(주)피더스이앤에스 완산충전소', 141)]

In [48]:
유초중고_count_list=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')
유초중고_count_list

[('(주)피더스이앤에스 완산충전소', 62),
 ('한옥마을주유소', 59),
 ('진영주유소', 57),
 ('SK강변주유소', 57),
 ('스타주유소', 56),
 ('전주제일주유소', 56),
 ('삼천주유소', 56),
 ('삼성Self주유소', 55),
 ('청솔주유소', 55),
 ('SK에너지㈜ 코끼리주유소', 55)]